In [405]:
import qsharp.azure

targets = qsharp.azure.connect(
            resourceId = "/subscriptions/6e59a4f1-00d8-48eb-8a3b-5127c7b07bd9/resourceGroups/AzureQuantum/providers/Microsoft.Quantum/Workspaces/BWSI-1",
            location = "eastus")

Connecting to Azure Quantum...

Connected to Azure Quantum workspace BWSI-1 in location eastus.


In [406]:
import qsharp

print("This workspace's targets:")
for target in targets:
    print("-", target.id)

This workspace's targets:
- ionq.qpu
- ionq.qpu.aria-1
- ionq.simulator
- microsoft.estimator
- quantinuum.hqs-lt-s1
- quantinuum.hqs-lt-s1-apival
- quantinuum.hqs-lt-s2
- quantinuum.hqs-lt-s2-apival
- quantinuum.hqs-lt-s1-sim
- quantinuum.hqs-lt-s2-sim
- quantinuum.qpu.h1-1
- quantinuum.sim.h1-1sc
- quantinuum.qpu.h1-2
- quantinuum.sim.h1-2sc
- quantinuum.sim.h1-1e
- quantinuum.sim.h1-2e
- rigetti.sim.qvm
- rigetti.qpu.aspen-m-2
- rigetti.qpu.aspen-m-3


In [407]:
runQAOA: any = None

In [408]:
import qsharp

In [419]:
%%qsharp


open Microsoft.Quantum.Canon;
open Microsoft.Quantum.Intrinsic;
open Microsoft.Quantum.Convert;
open Microsoft.Quantum.Math;
open Microsoft.Quantum.Arithmetic;
open Microsoft.Quantum.Preparation;
open Microsoft.Quantum.Diagnostics;

operation getGraphLength():Int{
    return 4;
}

operation completeCycle(gamma: Double, beta: Double, p: Int): Double {
    return meanCostInverse(QAOA(gamma, beta, p));
}

operation getGraphWeigths():Double[][]{
    //return [[0.,100.,0.,0.], [100.,0.,0.,0.], [0.,0.,0.,0.], [0.,0.,0.,0.]];
    return [[0., 1., 1., 1.],
 [1., 0., 1., 0.],
 [1., 1., 0., 1.],
 [1., 0., 1., 0.]];
}


operation QAOA(gamma:Double, beta:Double, p:Int):Int[]{
    
    //input register
    use register = Qubit[getGraphLength()];
    
    //prepare qubits by creating uniform superposition
    ApplyToEach(H, register);

    //repeat cost and mixer steps p times
    //I think that we might need a new gamma and beta for each p
    for i in 1..p{
        costLayer(register, gamma);
        mixerLayer(register, beta);
    }

    //Read the results as int array
    mutable res = new Int[Length(register)];
    for i in 0..Length(register)-1{
        set res w/= i <- (M(register[i]) == One ? 1 | 0);
    }

    //return the results
    return res;
}

operation mixerLayer(register:Qubit[], beta:Double):Unit{
    ApplyToEach(Rx(2.*beta, _), register); //the '2*beta' is from the qiskit vid https://www.youtube.com/watch?v=YpLzSQPrgSc
}
    
operation costLayer(register:Qubit[], gamma:Double):Unit{
    ApplyToEach(Rz(gamma, _), register); //should be weighted I think but idk how
    let weights = getGraphWeigths();
    for i in 0..Length(register)-1{
         for j in i+1..Length(register)-1{
             Rzz(gamma*weights[i][j], register[i], register[j]);
         }
     }
}
   
operation meanCostInverse(register: Int[]) : Double {
    mutable cost = 0.0;
    for i in 0 .. Length(register) - 1 {
        for j in i..Length(register)-1{
            let xi = IntAsDouble(register[i]);
            let xj = IntAsDouble(register[j]);
            set cost -= (1.-xi*xj)/2.; //from https://www.youtube.com/watch?v=aFVnWq3RHYU&t=126s
        }
    }
    return cost;// / IntAsDouble(Length(register));
}

operation meanCostInverse2(register: Int[]) : Double {
    mutable cost = 0.0;
    let weights = getGraphWeigths();
    for i in 0 .. Length(register) - 1 {
        for j in i..Length(register)-1{
            let xi = IntAsDouble(register[i]);
            let xj = IntAsDouble(register[j]);
            if (xi!=xj){
                set cost -=weights[i][j] ;
            }
        }
    }
    return cost;// / IntAsDouble(Length(register));
}


//-----------------------------------------------------------------------------------------------------
//Azure stuff

operation QAOAAzure(gamma:Double, beta:Double, p:Int):Result[]{
    
    //input register
    use register = Qubit[getGraphLength()];
    
    //prepare qubits by creating uniform superposition
    ApplyToEach(H, register);

    //repeat cost and mixer steps p times
    //I think that we might need a new gamma and beta for each p
    for i in 1..p{
        costLayer(register, gamma);
        mixerLayer(register, beta);
    }

    //Read the results as int array
    mutable res = new Result[Length(register)];
    for i in 0..Length(register)-1{
        set res w/= i <- M(register[i]);
    }

    //return the results
    return res;
}


D:\snippet_.qs(44,19): warning QS3308: Deprecated syntax. Use [] to construct an empty array, or [x, size = n] to construct an array of x repeated n times.
D:\snippet_.qs(114,19): warning QS3308: Deprecated syntax. Use [] to construct an empty array, or [x, size = n] to construct an array of x repeated n times.
D:\snippet_.qs(46,27): warning QS5023: The target Unspecified does not support comparing measurement results.


In [421]:
%%qsharp
@EntryPoint()
operation runQAOA():Result[]{
    let x = QAOAAzure(1.,1.,1);
    return x;
}

In [423]:
# # Set the target to ionq.simulator
# qsharp.azure.target("ionq.simulator")

# # We'll use 100 shots (simulated runs). Timeout is in seconds.
# result = qsharp.azure.execute(runQAOA, shots=100, jobName="run QAOA", timeout=240)

In [424]:
for i in range(10):
    print(QAOA.simulate(gamma=1, beta=1, p=10))

[0, 0, 0, 1]
[0, 0, 0, 1]
[0, 1, 0, 1]
[0, 0, 0, 0]
[1, 1, 1, 1]
[0, 1, 0, 1]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 1, 0, 0]
[0, 0, 0, 0]


In [425]:
import numpy as np
from scipy.optimize import minimize

In [426]:
def obj_func(params):
    myres=completeCycle.simulate(gamma=params[0], beta=params[1], p=1);
    print('res= ', myres)
    print('params= ', params)


In [427]:
def getMostFrequent(gamma, beta):
    myDict = {}
    for i in range(1000):
        res = QAOA.simulate(gamma=gamma, beta=beta, p=1)
        res=tuple(res)
        if res in myDict.keys():
            myDict.update({res:myDict.get(res)+1})
        else:
            myDict.update({res:0})
    return np.array(max(myDict))

In [428]:
print(getMostFrequent(11.,0.))

[1 1 1 1]


In [429]:
def compute_expectation(counts):
    """Computes expectation value based on measurement results
    Args:
        counts: (dict) key as bit string, val as count
        graph: networkx graph
    Returns:
        avg: float
             expectation value
    """
    avg = 0
    sum_count = 0
    for bit_string, count in counts.items():
        obj = meanCostInverse2.simulate(register=np.array(bit_string))
        #print(obj, ' =? ', meanCostInverse.simulate(register=np.array(bit_string)))
        avg += obj * count
        sum_count += count
    return avg/sum_count

In [438]:
def getFrequencyDict(gamma, beta):
    myDict = {}
    for i in range(100):
        res = QAOA.simulate(gamma=gamma, beta=beta, p=1)
        res=tuple(res)
        if res in myDict.keys():
            myDict.update({res:myDict.get(res)+1})
        else:
            myDict.update({res:0})
        #res=np.array(res)
    print(np.array(max(myDict, key=myDict.get)))
    #print(myDict)
    return myDict

In [439]:
print(compute_expectation(getFrequencyDict(1.,1.)))

[0 0 0 0]
-2.2386363636363638


In [440]:
def obj_func4(params):
    return compute_expectation(getFrequencyDict(params[0],params[1]))

In [441]:
res = minimize(obj_func4,[1.0, 1.0], method='COBYLA')
print('all= ', res)

[0 0 0 0]
[1 0 1 0]
[0 1 0 1]
[1 0 1 0]
[1 0 1 0]
[0 0 0 1]
[1 0 1 0]
[0 0 0 0]
[1 0 1 0]
[1 0 1 0]
[1 0 1 0]
[1 0 1 0]
[1 0 1 0]
[1 0 1 0]
[1 0 1 0]
[1 0 1 0]
[1 0 1 0]
[1 0 1 0]
[1 0 1 0]
[1 0 1 0]
[1 0 1 0]
[1 0 1 0]
[1 0 1 0]
[1 0 1 0]
[1 0 1 0]
[1 0 1 0]
[1 0 1 0]
[1 0 1 0]
[1 0 1 0]
[1 0 1 0]
all=       fun: -3.0476190476190474
   maxcv: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 30
  status: 1
 success: True
       x: array([2.41083562, 0.65321836])


In [251]:
def obj_func2 (params):
    myregister=getMostFrequent(params[0], params[1])
    print('params= ', params)
    print('register= ', myregister)
    myRes=meanCostInverse.simulate(register=myregister)
    print('cost= ', myRes)
    return myRes

In [252]:
print(obj_func2([2.,2.]))

params=  [2.0, 2.0]
register=  [1 1 1 1]
cost=  1.0
1.0


In [253]:
res = minimize(obj_func2,[0.0, 0.0], method='COBYLA')
print('all= ', res)

params=  [0. 0.]
register=  [1 1 1 1]
cost=  1.0
params=  [1. 0.]
register=  [1 1 1 1]
cost=  1.0
params=  [0. 1.]
register=  [1 1 1 1]
cost=  1.0
params=  [0.125 0.   ]
register=  [1 1 1 1]
cost=  1.0
params=  [0.    0.125]
register=  [1 1 1 1]
cost=  1.0
params=  [0.015625 0.      ]
register=  [1 1 1 1]
cost=  1.0
params=  [0.       0.015625]
register=  [1 1 1 1]
cost=  1.0
params=  [0.00195312 0.        ]
register=  [1 1 1 1]
cost=  1.0
params=  [0.         0.00195312]
register=  [1 1 1 1]
cost=  1.0
params=  [0.00024414 0.        ]
register=  [1 1 1 1]
cost=  1.0
params=  [0.         0.00024414]
register=  [1 1 1 1]
cost=  1.0
params=  [5.e-05 0.e+00]
register=  [1 1 1 1]
cost=  1.0
params=  [0.e+00 5.e-05]
register=  [1 1 1 1]
cost=  1.0
all=       fun: 1.0
   maxcv: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 13
  status: 1
 success: True
       x: array([0., 0.])


In [254]:
def obj_func3 (params):
    myregister=QAOA.simulate(gamma=params[0], beta=params[1], p=1)
    #print('params= ', params)
    #print('register= ', myregister)
    myRes=meanCostInverse.simulate(register=myregister)
    print('cost= ', myRes)
    return myRes

In [255]:
print(obj_func3([2.,2.]))

cost=  1.0
1.0


In [256]:
for i in range(5):
    res = minimize(obj_func3,[i, i+3.0], method='COBYLA')
    print('i= ',i,' i+3= ', i+3)
    print('all= ', res)

cost=  -0.25
cost=  0.875
cost=  -0.25
cost=  -0.25
cost=  -0.25
cost=  -0.25
cost=  1.0
cost=  -0.25
cost=  -0.25
cost=  0.125
cost=  -0.25
cost=  -0.25
cost=  1.0
cost=  0.5
cost=  0.5
cost=  -0.25
cost=  -0.25
cost=  -0.25
cost=  -0.25
i=  0  i+3=  3
all=       fun: -0.25
   maxcv: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 19
  status: 1
 success: True
       x: array([0., 3.])
cost=  1.0
cost=  0.5
cost=  1.0
cost=  -0.25
cost=  1.0
cost=  -0.25
cost=  -0.25
cost=  0.875
cost=  0.125
cost=  -0.25
cost=  -0.25
cost=  0.0
cost=  -0.25
cost=  0.625
cost=  -0.25
cost=  -0.25
cost=  0.875
cost=  0.125
cost=  -0.25
cost=  0.875
cost=  0.625
cost=  -0.25
cost=  -0.25
cost=  0.5
cost=  0.625
cost=  0.375
cost=  -0.25
i=  1  i+3=  4
all=       fun: -0.25
   maxcv: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 27
  status: 1
 success: True
       x: array([2.70719847, 3.29285332])
cost=  -0.25
cost=  -0.25
cost=  -0.25
cost=  0.125
cost=  -0.25
cost=  

In [398]:
from . import QAOA

ImportError: attempted relative import with no known parent package

In [397]:
QAOA.estimate_resources(
    JWEncodedData=encoded_data_caffeine,
    gamma=0.0,
    beta=-0.0,
    p=1,
)

AttributeError: 'QSharpCallable' object has no attribute 'estimate_resources'